In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
import folium
from collections import OrderedDict
from ipygee import *
import eeconvert
import pandas as pd
import numpy as np
# import geetools
# from google.colab import files

In [6]:
path_shp = "users/frandak2/bank/poligons_bs_test"
ROI = ee.FeatureCollection(path_shp)

In [2]:
def maskS2clouds(img):
# '''
# Function to mask clouds based on the pixel_qa band of Landsat 8 SR data. See:
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR

# Params:
# -------
# - img: image input Landsat 8 SR image

# Return:
# -------
# cloudmasked Landsat 8 image
# '''
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = img.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
           .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)

def Gap_fill(img):
    img_fill = img.focal_mean(1, 'square', 'pixels', 8)
    final_image = img_fill.blend(img)
    return final_image

Map = emap.Map(center=(40, -100), zoom=4)

In [7]:
# y = range(2011,2020)
# M = range(1,8)
# d = [1,28,30,31]
start_date = ee.Date.fromYMD(2011,1,1)
end_date = ee.Date.fromYMD(2011,1,31)
# 2.4.3 Prepare the satellite image (Landsat-8)
RGB_bands = ['B4','B3','B2'] #RGB
NDVI_bands = ['B5','B4'] #NIR

l7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")\
               .filterBounds(ROI)\
               .filterDate(start_date, end_date)\
               .map(maskS2clouds)\
               .median()\
               .multiply(0.0001)

l7_ndvi = l7.normalizedDifference(NDVI_bands).rename(['NDVI'])
l7_rgb = l7.select(RGB_bands).rename(['R','G','B']) 
l7 = l7_rgb.addBands(l7_ndvi)

In [8]:
# // Display the composites.
center = ROI.geometry().centroid().getInfo()['coordinates']
center.reverse()
# Create a visualization with folium
visParams_l7 = {    
  'bands': ['R', 'G', 'B'],
  'min': 0,
  'max': 0.5,
  'gamma': 1.4,
}

l7Mapid = l7.getMapId(visParams_l7)
Map.addLayer(l7, l7Mapid, 'L7', True, 0.5)
# Mapdisplay(center,l7Mapid,zoom_start=8)


In [ ]:
df2.to_csv('filename.csv') 
files.download('filename.csv')